In [1]:
import sys
import os

# 'JointYard'의 루트 디렉토리를 sys.path에 추가
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import pandas as pd
import numpy as np
import cv2 
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.animation import FFMpegWriter
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import plotly.graph_objects as go
# 가상환경에 pip install plotly 와 pip install nbformat 필요

from preprocess.loadLPdata import LoadLPData
from preprocess.reshapedata import reshape_lpdata_for_pca, pcadata_to_lpdata
from analysis.dim_reduction.pca import perform_pca 


### FUNCTION

In [2]:
def calculate_additional_keypoints(data, additional_points):
    """
    중점 키포인트를 계산하여 데이터에 추가합니다.
    
    Args:
        data (numpy.ndarray): (n_frames, n_keypoints * 3) 형태의 3D 키포인트 데이터.
        additional_points (dict): 중점을 계산할 키포인트 쌍과 이름. 
                                  예: {"12": (2, 3), "13": (1, 12)}.
    
    Returns:
        numpy.ndarray: 추가된 중점 키포인트를 포함한 데이터. (n_frames, (n_keypoints + 추가 키포인트) * 3)
    """
    n_frames, n_coords = data.shape
    n_keypoints = n_coords // 3
    data = data.reshape(n_frames, n_keypoints, 3)  # (n_frames, n_keypoints, 3)
    
    # 중점 계산을 위한 데이터 복사
    keypoint_dict = {i + 1: data[:, i] for i in range(n_keypoints)}  # 기존 키포인트
    new_keypoints = {}  # 추가 키포인트 저장
    
    for name, (kp1, kp2) in additional_points.items():
        if kp1 in keypoint_dict and kp2 in keypoint_dict:
            # 중점 계산
            midpoint = (keypoint_dict[kp1] + keypoint_dict[kp2]) / 2
            new_keypoints[int(name)] = midpoint
        else:
            raise ValueError(f"키포인트 {kp1} 또는 {kp2}가 존재하지 않습니다. 순서를 확인하세요.")
        
        # 새로 추가된 키포인트를 keypoint_dict에 저장 (다음 중점 계산에 사용 가능)
        keypoint_dict[int(name)] = new_keypoints[int(name)]
    
    # 기존 키포인트와 추가된 키포인트 결합
    additional_data = np.stack(new_keypoints.values(), axis=1) if new_keypoints else np.empty((n_frames, 0, 3))
    combined_data = np.concatenate([data, additional_data], axis=1)  # (n_frames, 기존 + 추가 키포인트, 3)
    
    return combined_data.reshape(n_frames, -1)  # (n_frames, (기존 + 추가 키포인트) * 3)


In [3]:
def rotate_data(data, angles_deg, axes=('x', 'y', 'z')):
    """
    3D 데이터를 지정된 축(axes) 및 각도(angles_deg)만큼 회전합니다.
    입력 데이터가 (n_frames, n_keypoints * 3) 형태일 경우 자동으로 처리합니다.
    
    Args:
        data (numpy.ndarray): (n_frames, n_keypoints * 3) 형태의 3D 데이터.
        angles_deg (list or tuple): 각 축에 대한 회전 각도 (도 단위).
        axes (list or tuple): 회전할 축. ('x', 'y', 'z') 순서로 지정.
    
    Returns:
        rotated_data (numpy.ndarray): 회전된 3D 데이터. 원래 형태 유지.
    """
    if len(angles_deg) != len(axes):
        raise ValueError("angles_deg와 axes의 길이는 같아야 합니다.")
    
    # 데이터를 (n_frames, n_keypoints, 3) 형태로 변환
    n_frames, n_coords = data.shape
    n_keypoints = n_coords // 3
    data_reshaped = data.reshape(n_frames, n_keypoints, 3)
    
    # 초기 회전 행렬 (단위 행렬)
    rotation_matrix = np.eye(3)
    
    # 각 축에 대한 회전 행렬 생성 및 조합
    for angle_deg, axis in zip(angles_deg, axes):
        angle_rad = np.deg2rad(angle_deg)
        if axis == 'x':
            axis_rotation = np.array([
                [1, 0, 0],
                [0, np.cos(angle_rad), -np.sin(angle_rad)],
                [0, np.sin(angle_rad), np.cos(angle_rad)],
            ])
        elif axis == 'y':
            axis_rotation = np.array([
                [np.cos(angle_rad), 0, np.sin(angle_rad)],
                [0, 1, 0],
                [-np.sin(angle_rad), 0, np.cos(angle_rad)],
            ])
        elif axis == 'z':
            axis_rotation = np.array([
                [np.cos(angle_rad), -np.sin(angle_rad), 0],
                [np.sin(angle_rad), np.cos(angle_rad), 0],
                [0, 0, 1],
            ])
        else:
            raise ValueError("Invalid axis. Choose from 'x', 'y', or 'z'.")
        
        # 회전 행렬 조합 (곱셈)
        rotation_matrix = np.dot(rotation_matrix, axis_rotation)
    
    # 데이터에 회전 행렬 적용
    rotated_data = np.zeros_like(data_reshaped)
    for frame_idx in range(n_frames):
        rotated_data[frame_idx] = np.dot(data_reshaped[frame_idx], rotation_matrix.T)
    
    # 원래 데이터 형태로 복구
    rotated_data_flattened = rotated_data.reshape(n_frames, n_coords)
    print("Complete data rotation")
    return rotated_data_flattened


In [4]:
def visualize_3d_keypoints(data, keypoint_pairs, output_path="output_video.mp4"):
    """
    3D 키포인트와 지정된 연결선을 시각화하여 동영상을 생성합니다.
    
    Args:
        data (numpy.ndarray): (n_frames, n_keypoints * 3) 형태의 3D 키포인트 데이터.
        keypoint_pairs (list of tuple): 연결할 키포인트 쌍 리스트. 예: [(1, 2), (1, 3), (4, 12)].
        output_path (str): 저장할 동영상 경로.
    """   
    n_frames, n_coords = data.shape
    n_keypoints = n_coords // 3
    data = data.reshape(n_frames, n_keypoints, 3) # (n_frames, n_keypoints, 3)

    # 데이터의 전체 범위 계산
    x_min, x_max = data[:, :, 0].min(), data[:, :, 0].max()
    y_min, y_max = data[:, :, 1].min(), data[:, :, 1].max()
    z_min, z_max = data[:, :, 2].min(), data[:, :, 2].max()

    margin = 0.1  # 범위 여유 (10%)
    x_range = (x_min - margin, x_max + margin)
    y_range = (y_min - margin, y_max + margin)
    z_range = (z_min - margin, z_max + margin)

    # 설정: 3D 플롯
    fig = plt.figure()
    ax = fig.add_subplot(111, projection="3d")

    # 선 생성
    colors = plt.cm.tab10(np.linspace(0, 1, len(keypoint_pairs)))
    lines = []
    for color in colors:
        line, = ax.plot([], [], [], color=color)
        lines.append(line)

    # 동영상 저장 설정
    writer = FFMpegWriter(fps=30, metadata=dict(artist="Me"), bitrate=1800)
    
    with writer.saving(fig, output_path, dpi=100):
        for frame_idx in range(n_frames):
            # 프레임 데이터
            frame_data = data[frame_idx]
            
            # 선 그리기
            for line, (start, end) in zip(lines, keypoint_pairs):
                start_point = frame_data[start - 1]
                end_point = frame_data[end - 1]
                
                # 각 선의 좌표 업데이트
                line.set_data([start_point[0], end_point[0]], [start_point[1], end_point[1]])
                line.set_3d_properties([start_point[2], end_point[2]])

            # 축 고정
            ax.set_xlim(*x_range)
            ax.set_ylim(*y_range)
            ax.set_zlim(*z_range)

            # 축 설정
            ax.set_title(f"Frame {frame_idx+1}")
            ax.set_xlabel("X")
            ax.set_ylabel("Y")
            ax.set_zlabel("Z")
            ax.grid(False)  # 그리드 제거
            #ax.axis('off')

            # 3D 뷰 회전 설정
            #ax.view_init(elev=180, azim=0) # elev: 위에서 바라보는 각도 (높이), azim: 시계 방향 회전 각도

            # 프레임 추가
            writer.grab_frame()
            
    plt.close(fig)


In [5]:
def visualize_3d_keypoints_and_video(data, keypoint_pairs, video_path, output_path="output_video.mp4"):
    """
    3D 키포인트와 지정된 연결선을 시각화하여 동영상을 생성합니다.
    비디오 프레임과 3D 키포인트를 나란히 시각화하여 저장합니다.
    
    Args:
        data (numpy.ndarray): (n_frames, n_keypoints * 3) 형태의 3D 키포인트 데이터.
        keypoint_pairs (list of tuple): 연결할 키포인트 쌍 리스트. 예: [(1, 2), (1, 3), (4, 12)].
        video_path (str): 비디오 파일 경로.
        output_path (str): 저장할 동영상 경로.
    """   
    n_frames, n_coords = data.shape
    n_keypoints = n_coords // 3
    data = data.reshape(n_frames, n_keypoints, 3)  # (n_frames, n_keypoints, 3)

    # 데이터의 전체 범위 계산
    x_min, x_max = data[:, :, 0].min(), data[:, :, 0].max()
    y_min, y_max = data[:, :, 1].min(), data[:, :, 1].max()
    z_min, z_max = data[:, :, 2].min(), data[:, :, 2].max()

    margin = 0.1  # 범위 여유 (10%)
    x_range = (x_min - margin, x_max + margin)
    y_range = (y_min - margin, y_max + margin)
    z_range = (z_min - margin, z_max + margin)

    # OpenCV를 사용해 비디오 파일을 읽어옵니다.
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: 비디오 파일을 열 수 없습니다.")
        return

    colors = plt.cm.tab10(np.linspace(0, 1, len(keypoint_pairs)))

    # 설정: 3D 플롯과 비디오 플롯을 나란히 배치
    fig = plt.figure(figsize=(12, 6))
    ax1 = fig.add_subplot(121)  # 비디오 프레임
    ax2 = fig.add_subplot(122, projection='3d')  # 3D 키포인트

    # 동영상 저장 설정
    writer = FFMpegWriter(fps=30, metadata=dict(artist="Me"), bitrate=1800)
    
    # 동영상 저장
    with writer.saving(fig, output_path, dpi=100):
        for frame_idx in range(n_frames):
            # 비디오 프레임 읽기
            ret, frame = cap.read()
            if not ret:
                print("Error: 비디오 프레임을 읽을 수 없습니다.")
                break

            # 이전 플롯 내용 지우기
            ax1.clear()
            ax2.clear()

            # 비디오 프레임 표시
            ax1.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))  # OpenCV는 BGR로 읽으므로 RGB로 변환
            ax1.axis('off')

            # 3D 키포인트 데이터 그리기
            frame_data = data[frame_idx]

            # 키포인트 연결선 그리기
            for start, end in keypoint_pairs:
                start_point = frame_data[start - 1]  # 0-index
                end_point = frame_data[end - 1]  # 0-index

                ax2.plot(
                    [start_point[0], end_point[0]],
                    [start_point[1], end_point[1]],
                    [start_point[2], end_point[2]],
                    label=f"{start}-{end}"
                )

            # 축 고정
            ax2.set_xlim(*x_range)
            ax2.set_ylim(*y_range)
            ax2.set_zlim(*z_range)

            # 축 설정
            ax2.set_title(f"Frame {frame_idx + 1}")
            ax2.set_xlabel("X")
            ax2.set_ylabel("Y")
            ax2.set_zlabel("Z")
            ax2.grid(False)

            # 프레임 추가
            writer.grab_frame()

    cap.release()
    plt.close(fig)


In [6]:
def save_3d_keypoints_frame(data, keypoint_pairs, frame_idx, output_path="output_frame.png"):
    """
    3D 키포인트와 지정된 연결선을 시각화하여 지정된 프레임의 이미지를 저장합니다.
    
    Args:
        data (numpy.ndarray): (n_frames, n_keypoints * 3) 형태의 3D 키포인트 데이터.
        keypoint_pairs (list of tuple): 연결할 키포인트 쌍 리스트. 예: [(1, 2), (1, 3), (4, 12)].
        frame_idx (int): 저장할 프레임의 인덱스 (0부터 시작).
        output_path (str): 저장할 이미지 경로.
    """   
    n_frames, n_coords = data.shape
    n_keypoints = n_coords // 3
    data = data.reshape(n_frames, n_keypoints, 3)  # (n_frames, n_keypoints, 3)

    # 데이터의 전체 범위 계산
    x_min, x_max = data[:, :, 0].min(), data[:, :, 0].max()
    y_min, y_max = data[:, :, 1].min(), data[:, :, 1].max()
    z_min, z_max = data[:, :, 2].min(), data[:, :, 2].max()

    margin = 0.1  # 범위 여유 (10%)
    x_range = (x_min - margin, x_max + margin)
    y_range = (y_min - margin, y_max + margin)
    z_range = (z_min - margin, z_max + margin)

    # 설정: 3D 플롯
    fig = plt.figure()
    ax = fig.add_subplot(111, projection="3d")

    # 선 생성
    colors = plt.cm.tab10(np.linspace(0, 1, len(keypoint_pairs)))
    lines = []
    for color in colors:
        line, = ax.plot([], [], [], color=color)
        lines.append(line)

    # 특정 프레임 데이터
    frame_data = data[frame_idx]
    
    # 선 그리기
    for line, (start, end) in zip(lines, keypoint_pairs):
        start_point = frame_data[start - 1]
        end_point = frame_data[end - 1]
        
        # 각 선의 좌표 업데이트
        line.set_data([start_point[0], end_point[0]], [start_point[1], end_point[1]])
        line.set_3d_properties([start_point[2], end_point[2]])

    # 축 고정
    ax.set_xlim(*x_range)
    ax.set_ylim(*y_range)
    ax.set_zlim(*z_range)

    # 축 설정
    ax.set_title(f"Frame {frame_idx+1}")
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")
    ax.grid(False)  # 그리드 제거

    # 3D 뷰 회전 설정 (필요시)
    # ax.view_init(elev=180, azim=0)

    # 이미지를 저장
    canvas = FigureCanvas(fig)
    canvas.print_figure(output_path, dpi=300)

    plt.close(fig)
    print(f"Saved frame {frame_idx+1} as image: {output_path}")


In [7]:
def plot_3d_keypoints_frame_plotly(data, keypoint_pairs, frame_idx):
    """
    3D 키포인트와 지정된 연결선을 Plotly로 시각화하여 지정된 프레임을 인터랙티브하게 화면에 표시합니다.
    
    Args:
        data (numpy.ndarray): (n_frames, n_keypoints * 3) 형태의 3D 키포인트 데이터.
        keypoint_pairs (list of tuple): 연결할 키포인트 쌍 리스트. 예: [(1, 2), (1, 3), (4, 12)].
        frame_idx (int): 표시할 프레임의 인덱스 (0부터 시작).
    """   
    # 데이터 준비
    n_frames, n_coords = data.shape
    n_keypoints = n_coords // 3
    data = data.reshape(n_frames, n_keypoints, 3)  # (n_frames, n_keypoints, 3)

    # 특정 프레임 데이터
    frame_data = data[frame_idx]
    
    # 키포인트 3D 위치
    x = frame_data[:, 0]
    y = frame_data[:, 1]
    z = frame_data[:, 2]

    # Plotly의 3D 산점도 (scatter3d)
    fig = go.Figure()

    # 3D 키포인트 플롯 (점의 크기 작게, 색을 검정으로)
    fig.add_trace(go.Scatter3d(
        x=x, y=y, z=z, mode='markers', marker=dict(size=2, color='black'),  # 크기 작고 색을 검정으로
        name="Keypoints"
    ))

    # 고유한 색상 목록 (20개의 색상을 지정)
    line_colors = [
        '#FF5733', '#33FF57', '#3357FF', '#F1C40F', '#9B59B6', '#1ABC9C',  # 1~6
        '#E74C3C', '#8E44AD', '#2ECC71', '#3498DB', '#F39C12', '#D35400',  # 7~12
        '#16A085', '#27AE60', '#2980B9', '#8E44AD', '#F39C12', '#9B59B6',  # 13~18
        '#34495E', '#7F8C8D'  # 19~20
    ]

    # 연결선 그리기 (선 두께를 더 두껍게)
    for idx, (start, end) in enumerate(keypoint_pairs):
        start_point = frame_data[start - 1]
        end_point = frame_data[end - 1]
        
        fig.add_trace(go.Scatter3d(
            x=[start_point[0], end_point[0]],
            y=[start_point[1], end_point[1]],
            z=[start_point[2], end_point[2]],
            mode='lines',
            line=dict(color=line_colors[idx % len(line_colors)], width=6),  # 색을 순차적으로 적용
            name=f"Line {start}-{end}"
        ))

    # 레이아웃 설정
    fig.update_layout(
        title=f"Frame {frame_idx + 1}",
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            aspectmode='cube',
            # 3D 공간 여백을 넉넉히 하기 위한 범위 설정 (더 넓게)
            xaxis_range=[x.min() - 300, x.max() + 300],
            yaxis_range=[y.min() - 300, y.max() + 300],
            zaxis_range=[z.min() - 300, z.max() + 300],
            bgcolor='rgba(0, 0, 0, 0)',  # 배경을 투명하게 설정
            xaxis=dict(
                showgrid=False, 
                showticklabels=False,  # 틱과 숫자 레이블 숨김
                showline=True,         # 축 선 표시
                linecolor='black',     # 축 선 색상 검정으로 설정
                linewidth=4,           # 축 선 두께 4로 설정
                backgroundcolor='rgba(169, 169, 169, 0.6)',  # XY 평면에 음영 주기
            ),
            yaxis=dict(
                showgrid=False, 
                showticklabels=False,  # 틱과 숫자 레이블 숨김
                showline=True,         # 축 선 표시
                linecolor='black',     # 축 선 색상 검정으로 설정
                linewidth=4,           # 축 선 두께 4로 설정
                backgroundcolor='rgba(169, 169, 169, 0.6)',  # YZ 평면에 음영 주기
            ),
            zaxis=dict(
                showgrid=False, 
                showticklabels=False,  # 틱과 숫자 레이블 숨김
                showline=True,         # 축 선 표시
                linecolor='black',     # 축 선 색상 검정으로 설정
                linewidth=4,           # 축 선 두께 4로 설정
                backgroundcolor='rgba(169, 169, 169, 0.6)',  # XZ 평면에 음영 주기
            ),
        ),
        showlegend=True
    )

    # Plotly로 인터랙티브한 플롯을 화면에 표시
    fig.show()


### DATALOADER

In [ ]:
# DATA LOAD INFO
base_file='/home/jiyoung/Desktop/lai_240524/Post-pose/example/debuglist/debuglist1121.xlsx'
eks_csv=True
n_keypoints=44
keypoints_name = [    
    "nose_1","l_ear_1","r_ear_1","body_1","l_fft_1","r_fft_1","l_hft_1","r_hft_1","top_tail_1","mid_tail_1","end_tail_1",
    "nose_2","l_ear_2","r_ear_2","body_2","l_fft_2","r_fft_2","l_hft_2","r_hft_2","top_tail_2","mid_tail_2","end_tail_2",
    "nose_3","l_ear_3","r_ear_3","body_3","l_fft_3","r_fft_3","l_hft_3","r_hft_3","top_tail_3","mid_tail_3","end_tail_3",
    "nose_4","l_ear_4","r_ear_4","body_4","l_fft_4","r_fft_4","l_hft_4","r_hft_4","top_tail_4","mid_tail_4","end_tail_4"
    ]
egocentric_alignment = False
egocentric_keypoint = None
save_dir = '/home/jiyoung/Desktop/lai_240524/Post-pose/result/debug7_validation/'
n_views=4

In [ ]:
loader = LoadLPData(base_file,False, n_keypoints, keypoints_name, save_dir, n_views)
lpdata_2d = loader.concatenated_df.groupby('SourceFile') # n_frames * 88

### 3D RECONSTRUCTION, related to LP multiview constrain

In [ ]:
print("Perform PCA with 3D Reconstruction")

pca_results=[]
sourcefile_indices = []
for sourcefile, lpdata_2d_group in lpdata_2d:
    # DataFrame에서 keypoints만 추출
    reshaped_lp_group = reshape_lpdata_for_pca(lpdata_2d_group, loader.keypoint_names, loader.n_views)  # (group_frames*11) * 4
    # PCA 수행
    pcadata, _, principal_components, _, _ = perform_pca(reshaped_lp_group, n_components=3)
    n_bodypoints=int(len(loader.keypoint_names)/loader.n_views)
    lpdata_3d_group = principal_components.reshape(len(principal_components)//n_bodypoints,3*n_bodypoints) # group_frames * 33
    # SourceFile 인덱스 기록
    sourcefile_indices.extend([sourcefile] * lpdata_3d_group.shape[0])
    pca_results.append(lpdata_3d_group)

lpdata_3d = np.vstack(pca_results) # n_frames * 33
sourcefile_indices = np.array(sourcefile_indices) 

### 2D LP DATA to 3D LP DATA RECONSTRUCTION

In [ ]:
## 2D data coordinate >> calibration estimation and triangulation

### CHESSBOARD CALIBRATION to 3D LP DATA RECONSTRUCTION

In [ ]:
## using chessboard video, in/extrinsic calibration >> 2D into 3D reconstruction

### VISUALIZE 3D MOIVE

In [ ]:
# Preparation of movie keypoint
additional_points = {"12": (2, 3)}
lpdata_3d_new = calculate_additional_keypoints(lpdata_3d, additional_points)

print(f"Original shape: {lpdata_3d.shape}")
print(f"New shape: {lpdata_3d_new.shape}")

In [ ]:
# Rotation of data for visualization
angles = [180]  # 각 축에 대한 회전 각도 (도 단위)
axes = ['x']  # 회전할 축
rotated_data = rotate_data(lpdata_3d_new, angles_deg=angles, axes=axes)
#np.save("/home/jiyoung/Desktop/lai_240524/Post-pose/result/debuglist2/rotated_data_x180.npy", rotated_data)

In [ ]:
#rotated_data = np.load("/home/jiyoung/Desktop/lai_240524/Post-pose/result/debuglist2/rotated_data_x180.npy")
#video_path = loader.video_paths # find video path

In [ ]:
# VISUALIE 3D MOVIE INFO 1
keypoint_pairs = [(1, 2), (1, 3), (2, 3), (4, 12), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (9, 10), (10, 11)]
specific_sourcefile = "4fused35_240626_124349_1440_1080.csv"
output_videopath = "/home/jiyoung/Desktop/lai_240524/Post-pose/result/debuglist2/3d_240626_124349.mp4"
#visualize_3d_keypoints(lpdata_3d_new[sourcefile_indices==specific_sourcefile], keypoint_pairs, output_videopath)
visualize_3d_keypoints(rotated_data[sourcefile_indices==specific_sourcefile], keypoint_pairs, output_videopath)

In [ ]:
# VISUALIE 3D MOVIE INFO 2
keypoint_pairs = [(1, 2), (1, 3), (2, 3), (4, 12), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (9, 10), (10, 11)]
specific_sourcefile2 = "4fused35_240625_152031_1440_1080.csv"
specific_video_path2 = "/home/jiyoung/Desktop/lai_240524/Post-pose/example/debuglist/4fused35_240625_152031_1440_1080.labeled.mp4"
output_videopath2 = "/home/jiyoung/Desktop/lai_240524/Post-pose/result/debuglist2/3d_video_240624_152031.mp4"
visualize_3d_keypoints_and_video(rotated_data[sourcefile_indices==specific_sourcefile2], keypoint_pairs, specific_video_path2, output_videopath2)

### Visualize Medoid Posture of Cluster

In [ ]:
# 사용 예시
medoid_indices = np.load('../path/to/medoid_indices.npy')
frame_idx=medoid_indices[0] # start with 0, input the cluster label number

keypoint_pairs = [(1, 2), (1, 3), (2, 3), (4, 12), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (9, 10), (10, 11)]
output_path = "/home/jiyoung/Desktop/lai_240524/Post-pose/result/medoid_cluster_0.png"
save_3d_keypoints_frame(rotated_data[sourcefile_indices==specific_sourcefile], keypoint_pairs, frame_idx, output_path)

In [ ]:
medoid_indices = np.load('/home/jiyoung/Desktop/lai_240524/Post-pose3D/result/debug7/hdbscan/medoid/medoid_indices_160.npy')
lpdata_3d = pd.read_csv('/home/jiyoung/Desktop/lai_240524/Post-pose3D/result/debug7/pca_0.99.csv').values

# Preparation of movie keypoint
additional_points = {"12": (2, 3)}
lpdata_3d_new = calculate_additional_keypoints(lpdata_3d, additional_points)

print(f"Original shape: {lpdata_3d.shape}")
print(f"New shape: {lpdata_3d_new.shape}")

# Rotation of data for visualization
angles = [180]  # 각 축에 대한 회전 각도 (도 단위)
axes = ['x']  # 회전할 축
rotated_data = rotate_data(lpdata_3d_new, angles_deg=angles, axes=axes)

# visualize medoid posture by clusters
keypoint_pairs = [(1, 2), (1, 3), (2, 3), (4, 12), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (9, 10), (10, 11)]
save_dir = '/home/jiyoung/Desktop/lai_240524/Post-pose3D/result/debug7/posture/160/'
os.makedirs(save_dir, exist_ok=True)
for idx, frame in enumerate(medoid_indices):
    output_path = os.path.join(save_dir, f'posture_cluster{idx}.png')
    save_3d_keypoints_frame(rotated_data,keypoint_pairs,frame,output_path)

In [8]:
medoid_indices = np.load('/home/jiyoung/Desktop/lai_240524/Post-pose3D/result/newdebug2/hdbscan_leaf/medoid/medoid_indices_500.npy')
lpdata_3d = pd.read_csv('/home/jiyoung/Desktop/lai_240524/Post-pose3D/result/newdebug2/pca_0.99.csv').values

# Preparation of movie keypoint
additional_points = {"12": (2, 3)}
lpdata_3d_new = calculate_additional_keypoints(lpdata_3d, additional_points)

print(f"Original shape: {lpdata_3d.shape}")
print(f"New shape: {lpdata_3d_new.shape}")

# Rotation of data for visualization
angles = [-30]  # 각 축에 대한 회전 각도 (도 단위)
axes = ['x']  # 회전할 축
rotated_data = rotate_data(lpdata_3d_new, angles_deg=angles, axes=axes)

# # visualize medoid posture by clusters
# keypoint_pairs = [(1, 2), (1, 3), (2, 3), (4, 12), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (9, 10), (10, 11)]
# save_dir = '/home/jiyoung/Desktop/lai_240524/Post-pose3D/result/newdebug2/hdbscan_leaf/posture/500/'
# os.makedirs(save_dir, exist_ok=True)
# for idx, frame in enumerate(medoid_indices):
#     plot_3d_keypoints_frame_plotly(lpdata_3d_new,keypoint_pairs,frame)
#     #plot_3d_keypoints_frame_plotly(rotated_data,keypoint_pairs,frame)
    

/home/jiyoung/anaconda3/envs/postpose2/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


Original shape: (499650, 33)
New shape: (499650, 36)
Complete data rotation


In [9]:
# closest_indices = np.load('/home/jiyoung/Desktop/lai_240524/Post-pose3D/result/newdebug2/hdbscan_leaf/medoid/cluster_sampling_indices_500.npy')
# medoid_frame = closest_indices[3,1]

In [ ]:
# visualize medoid posture by clusters
keypoint_pairs = [(1, 2), (1, 3), (2, 3), (4, 12), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (9, 10), (10, 11)]
save_dir = '/home/jiyoung/Desktop/lai_240524/Post-pose3D/result/newdebug2/hdbscan_leaf/posture/500/'
os.makedirs(save_dir, exist_ok=True)
cluster_number = 3
medoid_frame = medoid_indices[cluster_number]

plot_3d_keypoints_frame_plotly(lpdata_3d_new,keypoint_pairs,medoid_frame)
plot_3d_keypoints_frame_plotly(rotated_data,keypoint_pairs,medoid_frame)
    

In [40]:
#############plot closest indices in each cluster posture
medoid_indices = np.load('/home/jiyoung/Desktop/lai_240524/Post-pose3D/result/newdebug3/hdbscan/medoid/medoid_indices_500.npy')
lpdata_3d = pd.read_csv('/home/jiyoung/Desktop/lai_240524/Post-pose3D/result/newdebug2/pca_0.99.csv').values
closest_indices = np.load('/home/jiyoung/Desktop/lai_240524/Post-pose3D/result/newdebug3/hdbscan/medoid/cluster_sampling_indices_500.npy')

# Preparation of movie keypoint
additional_points = {"12": (2, 3)}
lpdata_3d_new = calculate_additional_keypoints(lpdata_3d, additional_points)

print(f"Original shape: {lpdata_3d.shape}")
print(f"New shape: {lpdata_3d_new.shape}")

# Rotation of data for visualization
angles = [40,20]  # 각 축에 대한 회전 각도 (도 단위)
axes = ['x','y']  # 회전할 축
rotated_data = rotate_data(lpdata_3d_new, angles_deg=angles, axes=axes)

# visualize medoid posture by clusters
keypoint_pairs = [(1, 2), (1, 3), (2, 3), (4, 12), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (9, 10), (10, 11)]

cluster_number = 3
medoid_frames = closest_indices[cluster_number][:6]
print(medoid_frames[0],medoid_indices[cluster_number])
for idx, frame in enumerate(medoid_frames):
    plot_3d_keypoints_frame_plotly(lpdata_3d_new,keypoint_pairs,frame)
    #plot_3d_keypoints_frame_plotly(rotated_data,keypoint_pairs,frame)
    

/home/jiyoung/anaconda3/envs/postpose2/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: FutureWarning:

arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.



Original shape: (499650, 33)
New shape: (499650, 36)
Complete data rotation
364813 364813
